# AJax 방식 가져오기

#### 페이지 소스에서 안 나올 때, 일부 화면 변경 형식의 데이터일 때
#### 개발자 도구 - 네트워크탭  - XHR -  request URL을 확인한다.
#### 확인이 가능하면 Ajax, 불가하면 Selenium
#### URL : https://sports.news.naver.com/kbaseball/schedule/index.nhn
#### Ajax : https://sports.news.naver.com/schedule/scoreBoard.nhn?date=20190317&category=kbo

In [ ]:
# Data structure

# 일별 데이터 누적 저장
# 1. csv file : 'KBO.csv' (8 variables)
# 'status', 'date', 'home_team', 'away_team',
# 'home_score','away_score','home_pitcher','away_pitcher'

# 중복 추출 방지를 위해 페이지 주소 저장
# 2. txt file : 'KBO_log.txt'
# urls 

In [112]:
import bs4, requests, time, os, re
import pandas as pd

In [113]:
# 서버에 페이지 요청하고 soup 반환하는 함수
def getSoup(url):
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

# 현재 주의 날짜 리스트를 반환하는 함수
def getWeek(soup):
    week_date = soup.select('ul.tab > li > a')[1:] # prev(지난 주) 제외
    week_date = week_date[:-1] # next(다음 주) 제외
    date = []
    for day in week_date:
        href = day.attrs['onclick']
        start = href.find('(')
        end = href.find(')')
        date.append(href[start+2:end-1])
    return date

# 다음 주의 첫 날짜를 반환하는 함수
def getNext(soup):
    next_date = soup.select('li.btn_next2 > a')[0].attrs['onclick']
    start = next_date.find('(')
    end = next_date.find(')')
    date = next_date[start+2:end-1]
    return date

In [114]:
# 해당 날짜(date)의 경기를 크롤링하는 함수
def getData(date):
    url = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={date}&category=kbo'
    
    # 파일에 요청했던 날짜의 url 기록하기
    log_filename = 'KBO_log.txt'
    # 파일이 없으면 생성
    exist_log = os.path.isfile(log_filename)
    if exist_log == False:  
        with open(log_filename, 'w') as fp:
            fp.write('')
    # 요청페이지 중복검사 
    with open(log_filename, 'r') as rfp:
        file = rfp.readlines()
        string = ' '.join(file)
        chk = string.find(url) # url 중복 검사
        if chk == -1:    # 중복되지 않은 페이지일 때 url 기록
            with open(log_filename, 'a') as wfp:
                str_url = f'{url}\n'
                wfp.writelines(str_url)
        else :   # 중복된 요청 페이지일 때 함수 종료
            print(f'{date} : exist')
            return
    
    # 크롤링 시작
    print(f'{date} : process')
    soup = getSoup(url)    
    ul_tag = soup.select('ul.sch_vs')
    data_all = []
    if len(ul_tag) > 0 :
        li_list = soup.select('ul.sch_vs > li')
        for li_tag in li_list:
            data_list = []  # 데이터 한 경기(행) 데이터
            class_value = li_tag.attrs['class']
            if class_value[0] == 'end' : # 0번째 index로 경기종료,이전 구분
                data_list.append('경기종료')
                data_list.append(date)
                # 팀 데이터 (home,away)
                team_tag = li_tag.select('p.vs_team > strong')
                data_list.append(team_tag[1].text.strip())
                data_list.append(team_tag[0].text.strip())
                # 점수 데이터 (home,away)
                team_score = li_tag.select('strong.vs_num')
                data_list.append(team_score[1].text.strip())
                data_list.append(team_score[0].text.strip())
                # 투수 데이터 (home, away)
                team_pitcher = li_tag.select('span.game_info > a')
                if len(team_pitcher) > 0 :
                    data_list.append(team_pitcher[1].text.strip())
                    data_list.append(team_pitcher[0].text.strip())
                else:  
                    data_list.append('모름')
                    data_list.append('모름')
                    
            elif class_value[0] == 'before_game':
                # 이전 경기 + 경기 취소 (경기취소는 크롤링 후 날짜로 판단 가능)
                data_list.append('경기전')
                data_list.append(date)
                # 팀 데이터 있음
                team_tag = li_tag.select('p.vs_team > strong')
                data_list.append(team_tag[1].text.strip())
                data_list.append(team_tag[0].text.strip())
                # 점수 데이터 없음
                data_list.append('-1')
                data_list.append('-1')
                # 투수 데이터 없음
                data_list.append('모름')
                data_list.append('모름')
            #if-elif end
            data_all.append(data_list)  # 한 경기(행)를 리스트에 추가
        #for end
    head_list = ['status', 'date', 'home_team', 'away_team', 
                 'home_score','away_score','home_pitcher','away_pitcher']
    df = pd.DataFrame(data_all, columns = head_list )
    filename = 'KBO.csv'    
    exists = os.path.isfile(filename)
    # if end
    
    if exists == False:  # 파일이 없으면 헤더 및 내용 추가
        df.to_csv(filename, index=False, header=True, encoding='utf-8-sig')
    else: # 파일이 있으면 내용 추가
        df.to_csv(filename, index=False, header=False, encoding='utf-8-sig',mode='a')

In [115]:
# 'start_date'부터 크롤링 시작
start_date = 20190311
url = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={start_date}&category=kbo'
i=0 # for Delay
while True:
    i += 1
    #if i == 5: break
    soup = getSoup(url)
    week = getWeek(soup) # date list
    for day in week:
        getData(day)
    
    next_week = getNext(soup)  # a date or None
    if len(next_week) == 0:
        print('process done')
        break
    else :
        url = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={next_week}&category=kbo'
        if (i % 3) == 0 : time.sleep(1) # m주 마다 딜레이

20190311 : process
20190312 : process
20190313 : process
20190314 : process
20190315 : process
20190316 : process
20190317 : process
20190318 : process
20190319 : process
20190320 : process
20190321 : process
20190322 : process
20190323 : process
20190324 : process
20190325 : process
20190326 : process
20190327 : process
20190328 : process
20190329 : process
20190330 : process
20190331 : process
20190401 : process
20190402 : process
20190403 : process
20190404 : process
20190405 : process
20190406 : process
20190407 : process
20190408 : process
20190409 : process
20190410 : process
20190411 : process
20190412 : process
20190413 : process
20190414 : process
20190415 : process
20190416 : process
20190417 : process
20190418 : process
20190419 : process
20190420 : process
20190421 : process
20190422 : process
20190423 : process
20190424 : process
20190425 : process
20190426 : process
20190427 : process
20190428 : process
20190429 : process
20190430 : process
20190501 : process
20190502 : p